In [107]:
import pandas as pd
import numpy as np
import scipy.interpolate as sci
from sklearn.metrics import mean_squared_error

# Data Functions

In [108]:
# read dataset from a path
def read_dataset_single(path):
    #returns a panda object with just the 3 columns Node, y, x
    df = pd.read_csv(path, sep='\t')
    df = df[['Node Name', 'Total-time', 'Data Size']]
    return df

# read dataset from a list of paths
def read_dataset_all(name_list):
    # returns a panda object with all datasets combined
    df_result = pd.DataFrame(['Node Name', 'Total-time', 'Data Size'])
    
    for path in name_list:
        df_specific_result = read_dataset_single(path)
        df_result = pd.concat([df_result, df_specific_result])
        
    return df_result


# build a cleaned dataset with a minimum number of nodes, determined by the threshold
def clean_data(df, threshold):
    
    nodename_counts = df['Node Name'].value_counts()
    valid_nodenames = nodename_counts[nodename_counts >= threshold].index.tolist()
    
    return df[df['Node Name'].isin(valid_nodenames)]


# returns array with all the unique nodenames
def give_nodenames(df):
    return np.unique(df['Node Name'].values)


# returns just the x and y values for a Node as np.arrays
# they are returned as sorted (by x) and unique
# sort values (for easier fitting and interpolation)
def give_x_and_y_per_node(df_general_result, name):

    df = df_general_result[df_general_result['Node Name'] == name]
    x, y = df['Data Size'].values, df['Total-time'].values
    
    unique_x, unique_indices = np.unique(x, return_index=True)
    unique_y = y[unique_indices]

    sorted_indices = np.argsort(unique_x)
    
    return unique_x[sorted_indices], unique_y[sorted_indices]

# Plotting Function

In [109]:
import matplotlib.pyplot as plt

def plot_data_and_fitted_functions(x_values, y_values, function_to_plot, titel_name):
    
    fig, ax = plt.subplots()

    # plot x,y
    ax.scatter(x_values, y_values, label="Data Points", color='b', marker='o')

    # create estimations for plotting the function
    x_range = np.linspace(min(x_values), max(x_values), 100)
    y_range = function_to_plot(x_range)

    ax.plot(x_range, y_range, label="function for estimation", color='r')

    ax.set_xlabel("X Values")
    ax.set_ylabel("Y Values")
    ax.set_title(titel_name)

    ax.legend()

    plt.show()

# Necessery Functions for Polyfit (poly1..4 and log1..4)

In [110]:
#for estimation of Polynominal
def polynominal_estimation(x, coefficients):
    return np.polyval(coefficients, x)

# Curvefitting is optimized for log and exp functions
# Polyfit is optimizied for polynominals
# def log_polynominal_estimation(x, coefficients):
#     return np.log(np.polyval(coefficients, x))

# # for curve fitting
# #--------polynominal----------
def poly_1_curve(x, a, b):
    return np.polyval([a,b], x)

def poly_2_curve(x, a, b, c):
    return np.polyval([a,b,c], x)

def poly_3_curve(x, a, b, c, d):
    return np.polyval([a,b,c,d], x)

def poly_4_curve(x, a, b, c, d, e):
    return np.polyval([a,b,c,d,e], x)

#----------log----------
def log_1_curve(x, a, b):
    return np.log(np.polyval([a,b], x))

def log_2_curve(x, a, b, c):
    return np.log(np.polyval([a,b,c], x))

def log_3_curve(x, a, b, c, d):
    return np.log(np.polyval([a,b,c,d], x))

def log_4_curve(x, a, b, c, d, e):
    return np.log(np.polyval([a,b,c,d,e], x))

# Fitting Functions

In [111]:
from scipy.optimize import curve_fit


#----------Fitting (Standart)----------
def polynomial_fit(x, y, degree):
    return np.polyfit(x, y, degree)

# # basically same function, but for better understanding here
# def log_polynominal_fitting(x, y_exp, degree):
#     return np.polyfit(x, y_exp, degree)

# deprecated Polynominal.fit is the new one
#

#----------Curve Fitting----------
# for large datasets, u should give a rough guess to speed up the fitting
# here we usually use the results from the previous interpolations
def curve_fitting_coefficients(x_values, y_values, function, guess=''):
    coeff, _ = curve_fit(function, x_values, y_values, p0=guess)
    return coeff

In [112]:
def make_log(x_values, y_values, results, group_of_methods, index):
    
    list_of_functions = [log_1_curve, log_2_curve, log_3_curve, log_4_curve]
    y_log = np.log(y_values)
    for i in group_of_methods['log']:
        coeff = np.polyfit(x=x_values, y=y_log, deg=i)
        
        results[index + 2*i-2] = coeff
        results[index + 2*i-1] = mean_squared_error(y_values, np.exp(np.polyval(coeff, x_values)))
        
        index += 2
        
        
    for i in group_of_methods['log_curve']:

        coeff = curve_fitting_coefficients(x_values, y_log, list_of_functions[i-1], results[2*i-2])
        results[index + 2*i-2] = coeff
        results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
        
        index += 2
        
    return index

In [113]:
def make_poly(x_values, y_values, results, group_of_methods, index):
    
    list_of_functions = [poly_1_curve, poly_2_curve, poly_3_curve, poly_4_curve]
    
    for i in group_of_methods['poly']:
        
        coeff = np.polyfit(x=x_values, y=y_values, deg=i)
        
        results[2*i-2] = coeff
        results[2*i-1] = mean_squared_error(y_values, np.polyval(coeff, x_values))
        index += 2
        
        
    for i in group_of_methods['poly_curve']:

        coeff = curve_fitting_coefficients(x_values, y_values, list_of_functions[i-1], results[2*i-2])
        results[index + 2*i-2] = coeff
        results[index + 2*i-1] = mean_squared_error(y_values, np.polyval(coeff, x_values))
        index += 2
        
    return index

# Spline Functions

In [114]:
import scipy.interpolate as sci
# more complicated than before, but the old stuff was deprecated and mostly legacy code
# here we can also give us the spline functions and how they look
# I implemented the fitting and the interpolation

# for Spline prediction we need some help values

# gives me t_knots as a regular intervall between start and endpoint with k+1 regularity
# if there are too few data points it just takes them in an uniform distance.
# def get_t_Knots_regular(x_values, k_degree):
    
#     r = (2 * k_degree + 1) if (len(x_values) > 2 * (k_degree + 1)) else len(x_values)
    
#     return np.linspace(x_values[0], x_values[-1], r)



# import UnivariateSpline
# scipy.interpolate import make_smoothing_spline
# for smoothing
def get_s_smoothing_approximation_1(y_values):
    return len(y_values) - np.sqrt(2 * len(y_values))

def get_s_smoothing_approximation_2(y_values):
    return len(y_values) * np.var(y_values)


#----------Spline with smoothing----------
# create spline object with smoothing build in (less knots)
# def spline_fitting_with_smoothing_1(x_values, y_values, degree, smooth):
#     return sci.UnivariateSpline(x_values, y_values, k=degree, s=smooth)

# needs weights or smoothing condition
# sci.splrep()

# need knots t
# sci.make_lsq_spline()
    
#----------Fitting----------

# make_smoothing_spline
# needs x,y
# make_smoothing_spline(x, y, w=None, lam=None)
# returns BSpline object

# relevant results for a spline
# Knots, coefficient, degree

# Smoothing Coniditions explanation
https://en.wikipedia.org/wiki/Smoothing_spline

In [115]:
def make_sspline(x_values, y_values, index, results):
    # make_smoothing_spline
    # needs x,y
    # make_smoothing_spline(x, y, w=None, lam=None)
    # returns BSpline object
    
    bspline = sci.make_smoothing_spline(x=x_values, y=y_values)
    index=-2
    results[index] = (bspline.tck)
    results[index+1] = mean_squared_error(y_values, bspline(x=x_values))
    
    return index

In [116]:
def make_uspline(x_values, y_values, index, results, group_of_methods):
    # univariantspline
    # needs x,y, w or s, k
    # UnivariateSpline(x, y, w=None, bbox=[None, None], k=3, s=None, ext=0, check_finite=False)
    # is an univariantspline object
    
    s1 = get_s_smoothing_approximation_1(y_values)
    s2 = get_s_smoothing_approximation_2(y_values)
    
    for i in group_of_methods['univariant_spline_2']:
        index -= 2
        uspline = sci.UnivariateSpline(x=x_values, y=y_values, s=s2, k=i)
        results[index] = (uspline.get_knots(), uspline.get_coeffs(), i)
        results[index+1] = mean_squared_error(y_values, uspline(x=x_values))
        
    for i in group_of_methods['univariant_spline_1']:
        index -= 2
        uspline = sci.UnivariateSpline(x=x_values, y=y_values, s=s1, k=i)
        results[index] = (uspline.get_knots(), uspline.get_coeffs(), i)
        results[index+1] = mean_squared_error(y_values, uspline(x=x_values))
        
    return index

In [117]:
def make_splinerep(x_values, y_values, index, results, group_of_methods):
    # splrep
    # needs x,y, w or s, k
    # splrep(x, y, w=None, xb=None, xe=None, k=3, task=0, s=None, t=None, full_output=0, per=0, quiet=1)
    # returns parameters for a bsplineobject
    
    s1 = get_s_smoothing_approximation_1(y_values)
    s2 = get_s_smoothing_approximation_2(y_values)
    
    for i in group_of_methods['spline_approximation_2']:
        index -= 2
        tck = sci.splrep(x=x_values, y=y_values, s=s2, k=i)
        
        results[index] = tck
        results[index+1] = mean_squared_error(y_values, sci.splev(x=x_values, tck=tck))
        

    for i in group_of_methods['spline_approximation_1']:
        index -= 2
        tck = sci.splrep(x=x_values, y=y_values, s=s1, k=i)
        results[index] = tck
        results[index+1] = mean_squared_error(y_values, sci.splev(x=x_values, tck=tck))
    
    return index

In [118]:
# returns a dict with the min_mse, best method, coefficients of the method and function of the method
def comparing_fitting_methods(x_values, y_values, group_of_methods, k):

    poly = False
    log = False
    poly_curve = False
    log_curve = False
    s_spline = False
    u_spline_1 = False
    u_spline_2 = False
    splrep_1 = False
    splrep_2 = False
    k0 = 0


    for key in group_of_methods.keys():
        poly |= key == 'poly'
        log |= key == 'log'
        poly_curve |= key == 'poly_curve'
        log_curve |= key == 'log_curve'
        s_spline |= key == 'smoothing_spline'
        u_spline_1 |= key == 'univariant_spline_1'
        u_spline_2 |= key == 'univariant_spline_2'
        splrep_1 |= key == 'spline_approximation_1'
        splrep_2 |= key == 'spline_approximation_2'


    results = [None] * k * 2

    if splrep_2 or splrep_1:
        k = make_splinerep(x_values, y_values, k, results, group_of_methods)
        
    if u_spline_1 or u_spline_2:
        k = make_uspline(x_values, y_values, k, results, group_of_methods)

    if s_spline:
        k = make_sspline(x_values, y_values, k, results)
        
    if poly or poly_curve:
        k0 = make_poly(x_values, y_values, results, group_of_methods, k0)
        
    if log or log_curve:
        k0 = make_log(x_values, y_values, results, group_of_methods, k0)
        
    print(k0, k)
    return results

In [119]:
def build_solution_overview_by_node(list_of_Paths, threshold, group_of_methods):
    df = read_dataset_all(list_of_Paths)
    df = clean_data(df, threshold)
    nodenames = give_nodenames(df)
    methods = []
    k=0
    
    for key, value in group_of_methods.items():
        k += len(value)*2
        for i in value:
            methods += [f"{key} {'_'} {i}"]
            methods += [f"{key} {'_'} {i} {'_mse'}"]

    # result_df = pd.DataFrame(index=nodenames)
    result_dict ={}
    for name in nodenames:
        x_sorted, y_sorted = give_x_and_y_per_node(df, name)
        result_dict[name] = comparing_fitting_methods(x_sorted, y_sorted, group_of_methods, k)
    
    df = pd.DataFrame(result_dict)
    df.to_csv('test.csv')
    print(result_dict)
    

In [120]:
# def build_solution_overview_by_genome(list_of_Paths, list_of_names, threshold, group_of_methods):
#     df = read_dataset_all(list_of_Paths)
#     df = clean_data(df, threshold)
#     nodenames = give_nodenames(df)
#     # col_names = ['Node Name'] + [('polynominal_'+str(i), 'coefficients','function', 'mse') for i in range(1,poly+1)] + [('log'+str(i), 'coefficients', 'function', 'mse') for i in range(1,log+1)] + [('polynominal_'+str(i), 'coefficients', 'function','mse') for i in range(1,spline+1)]
#     result_dict = {}
#     for name in nodenames:
#         x_sorted, y_sorted = give_x_and_y_per_node(df, name)
#         result_dict[name] = comparing_fitting_methods(x_sorted, y_sorted, group_of_methods)
    
#     result_df = pd.DataFrame(result_dict)
#     result_df = result_df.T
#     result_df.to_csv('test.tsv', sep='\t')
#     print(result_df)

# Necessary Constants

In [121]:
from pathlib import Path

path_finals = [Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_viral500m.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_archea1.4g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria30g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria58g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria88g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria125g.tsv")]

name_list = ["viral500m", "archea1.4g", "bacteria30g", "bacteria58", "bacteria88", "bacteria125"]

group_of_methods = {
        # define here what you want
        'poly' : [1,2,3], #polynominal fitting, with degrees
        'poly_curve' : [1,2,3], #polynominal curve fitting, with degrees
        'log' : [1,2,3], #logarithmic fitting, with degrees
        'log_curve': [1,2,3], #logarithmic curvefitting, with degrees
        'smoothing_spline' : [1], #smoothing spline, finds a smooth spline with a degree
        'univariant_spline_1': [1,2,3], #makes an univariant spline (same length of knots), with degree
        'univariant_spline_2': [1,2,3], #makes an univariant spline (same length of knots), with degree
        'spline_approximation_1': [1,2,3], #makes a spline approximation
        'spline_approximation_2': [1,2,3] #makes a spline approximation
    }

# univariantspline
# needs x,y, w or s, k
# UnivariateSpline(x, y, w=None, bbox=[None, None], k=3, s=None, ext=0, check_finite=False)
# is an univariantspline object

# splrep
# needs x,y, w or s, k
# splrep(x, y, w=None, xb=None, xe=None, k=3, task=0, s=None, t=None, full_output=0, per=0, quiet=1)
# returns parameters for a bsplineobject

# make_lsq_spline
# mneeds x,y, t, k
# make_lsq_spline(x, y, t, k=3, w=None, axis=0, check_finite=True)
# returns BSpline object

# make_smoothing_spline
# needs x,y
# make_smoothing_spline(x, y, w=None, lam=None)
# returns BSpline object

# relevant results for a spline
# Knots, coefficient, degree



threshold=10 #what is the minimum number of datapoints to consider per Node


build_solution_overview_by_node(path_finals, threshold, group_of_methods)

/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[

24 -2
24 -2
24 -2
24 -2


/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:29: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval([a,b,c], x))
/Users/manuez42/Library/Python/3.11/lib/python/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:32: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval(

24 -2


/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:26: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval([a,b], x))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:29: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval([a,b,c], x))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:32: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval([a,b,c,d], x))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipy

...

24 -2
24 -2


/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:26: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval([a,b], x))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:29: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval([a,b,c], x))
/Users/manuez42/Library/Python/3.11/lib/python/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x

24 -2
24 -2
24 -2


/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:26: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval([a,b], x))
/Users/manuez42/Library/Python/3.11/lib/python/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/282942124.py:18: RuntimeWarning: overflow encountered in exp
  results[index + 2*i-1] = np.exp(mean_squared_error(y_log, np.polyval(coeff, x_values)))
/var/folders/y3/dw3vl1gj7jbbzl70zgvxn_t1rdqhgf/T/ipykernel_13121/1496222104.py:32: RuntimeWarning: invalid value encountered in log
  return np.log(np.polyval([a

24 -2
{201.0: [array([-8.58585468e-03,  4.99075493e+02]), 197526.72009278592, array([-5.66524840e-04,  1.87030157e+00, -2.93159088e+02]), 34249.98971684803, array([-2.03212793e-07,  3.30205118e-04,  9.66855650e-01, -9.64794430e+01]), 31582.648659037106, array([-8.58585468e-03,  4.99075493e+02]), 197526.72009278592, None, None, array([-5.66524840e-04,  1.87030157e+00, -2.93159088e+02]), 34249.98971684803, array([-1.97471892e-04,  5.97825470e+00]), 228246.4019892943, array([-2.03212793e-07,  3.30205118e-04,  9.66855650e-01, -9.64794430e+01]), 31582.648659037106, array([-1.17101397e-06,  3.68621205e-03,  4.34069586e+00]), 33502.58316665119, array([-9.88892694e-02,  4.39919350e+02]), inf, array([-1.58058131e-10, -4.73540849e-07,  2.98351528e-03,  4.49367253e+00]), 39480.27301370669, array([-3.64696738e-04,  1.17145580e+00, -5.56508130e+01]), inf, None, None, array([-6.39854603e-07,  2.22980551e-03, -8.64595300e-01,  2.63570034e+02]), inf, (array([ 172.        ,  338.        ,  511.5       